In [6]:
#Partial Correlation between environmental and topographic drivers vs SOS and EOS
# topographic drivers: 1. elevation, 2. slope, aspect resolved into: 3. northness (cos theta) and 4. eastness (sin theta)


In [7]:
import numpy as np  
import pandas as pd
import rasterio as rio  
import xarray as xr
import rioxarray as rxr 
import os   
import geopandas as gpd
import matplotlib.pyplot as plt
import pingouin as pg

In [8]:
#import topographic factors rasters

ecoregion_rxr = rxr.open_rasterio(r"../Data/Ecoregion_raster/ecoregions_raster.tif")
elev_rxr = rxr.open_rasterio(r"../Data/DEM_Rasters/elevation.tif")
aspect_rxr = rxr.open_rasterio(r"../Data/DEM_Rasters/aspect.tif")
slope_rxr = rxr.open_rasterio(r"../Data/DEM_Rasters/slope.tif")

lsp_metrics = ['sos', 'eos']
base_outdir = r"../Data/Processed/Correlation_Analysis/Topographic_Factors/"
os.makedirs(base_outdir, exist_ok=True)

In [9]:
# for metric in lsp_metrics:
#     sig_trend_rxr = rxr.open_rasterio(r"../Data/Trend_Rasters/mod_" + metric + "_mk_significant.tif")
#     valid_lsp_rxr = rxr.open_rasterio(r"../Data/Processed/Valid_lsp_raster/Valid_lsp_change_" + metric + ".tif")

#     #mask sig_trend rxr to pixel locations where lsp change is valid
#     valid_trend_rxr = sig_trend_rxr.where(valid_lsp_rxr == 1)

#     aspect_rad = np.deg2rad(aspect_rxr)
#     #resolve aspect into northness and eastness
#     aspect_n_rxr = np.cos(aspect_rad)
#     aspect_e_rxr = np.sin(aspect_rad)

#     #reproject everything to valid_trend_rxr
#     raster_dict = {
#         "valid_trend": valid_trend_rxr,
#         "ecoregion": ecoregion_rxr,
#         "elevation": elev_rxr,
#         "slope": slope_rxr,
#         "aspect_n": aspect_n_rxr,
#         "aspect_e": aspect_e_rxr
#         }

#     aligned_rasters = []

#     for name, raster in raster_dict.items():
#         raster = raster.rio.write_crs("EPSG: 4326")
#         reproj = raster.rio.reproject_match(valid_trend_rxr)
#         reproj.name = name
#         reproj = reproj.squeeze('band', drop = True)
#         aligned_rasters.append(reproj)

#     stacked_xr = xr.merge(aligned_rasters, compat = 'override')
#     stacked_df = stacked_xr.to_dataframe().reset_index()
#     stacked_df = stacked_df[['x', 'y', 'valid_trend', 'ecoregion', 'elevation', 'slope', 'aspect_n', 'aspect_e']]
#     select_ecoregions = [81003, 40115, 40301, 40403, 40401, 40166, 81021, 40701, 40501, 40502, 40120]
#     stacked_df = stacked_df[stacked_df['ecoregion'].isin(select_ecoregions)]
#     stacked_df = stacked_df.dropna().reset_index(drop=True)

#     local_r = []
#     for ecoregion in select_ecoregions:
#         try:
#             corr_elev = pg.partial_corr(data=stacked_df[stacked_df['ecoregion'] == ecoregion], x='elevation', y='valid_trend', covar=['slope', 'aspect_n', 'aspect_e'])
#             corr_slope = pg.partial_corr(data=stacked_df[stacked_df['ecoregion'] == ecoregion], x='slope', y='valid_trend', covar=['elevation', 'aspect_n', 'aspect_e'])
#             corr_aspect_n = pg.partial_corr(data=stacked_df[stacked_df['ecoregion'] == ecoregion], x='aspect_n', y='valid_trend', covar=['elevation', 'slope', 'aspect_e'])
#             corr_aspect_e = pg.partial_corr(data=stacked_df[stacked_df['ecoregion'] == ecoregion], x='aspect_e', y='valid_trend', covar=['elevation', 'slope', 'aspect_n'])
#             local_r.append([ecoregion, corr_elev['r'].iloc[0], corr_slope['r'].iloc[0], corr_aspect_n['r'].iloc[0], corr_aspect_e['r'].iloc[0]])
#         except:
#             local_r.append([ecoregion, None, None, None, None])

#     r_topo_lsp = pd.DataFrame(local_r, columns=['ecoregion', 'elevation', 'slope', 'aspect_n', 'aspect_e']).round(3)
#     r_topo_lsp.to_csv(os.path.join(base_outdir, "partial_r_topographic_" + metric + ".csv"), index=False)
#     print("Correlations for " + metric + " saved to " + os.path.join(base_outdir, "correlations_topographic_" + metric + ".csv"))

In [11]:
from scipy.stats import spearmanr
for metric in lsp_metrics:
    sig_trend_rxr = rxr.open_rasterio(r"../Data/Trend_Rasters/mod_" + metric + "_mk_significant.tif")
    valid_lsp_rxr = rxr.open_rasterio(r"../Data/Processed/Valid_lsp_raster/Valid_lsp_change_" + metric + ".tif")

    #mask sig_trend rxr to pixel locations where lsp change is valid
    valid_trend_rxr = sig_trend_rxr.where(valid_lsp_rxr == 1)

    aspect_rad = np.deg2rad(aspect_rxr)
    #resolve aspect into northness and eastness
    aspect_n_rxr = np.cos(aspect_rad)
    aspect_e_rxr = np.sin(aspect_rad)

    #reproject everything to valid_trend_rxr
    raster_dict = {
        "valid_trend": valid_trend_rxr,
        "ecoregion": ecoregion_rxr,
        "elevation": elev_rxr,
        "slope": slope_rxr,
        "aspect_n": aspect_n_rxr,
        "aspect_e": aspect_e_rxr
        }

    aligned_rasters = []

    for name, raster in raster_dict.items():
        raster = raster.rio.write_crs("EPSG: 4326")
        reproj = raster.rio.reproject_match(valid_trend_rxr)
        reproj.name = name
        reproj = reproj.squeeze('band', drop = True)
        aligned_rasters.append(reproj)

    stacked_xr = xr.merge(aligned_rasters, compat = 'override')
    stacked_df = stacked_xr.to_dataframe().reset_index()
    stacked_df = stacked_df[['x', 'y', 'valid_trend', 'ecoregion', 'elevation', 'slope', 'aspect_n', 'aspect_e']]
    select_ecoregions = [81003, 40115, 40301, 40403, 40401, 40166, 81021, 40701, 40501, 40502, 40120]
    stacked_df = stacked_df[stacked_df['ecoregion'].isin(select_ecoregions)]
    stacked_df = stacked_df.dropna().reset_index(drop=True)

    local_r = []
    for ecoregion in select_ecoregions:
        try:
            ecoregion_df = stacked_df[stacked_df['ecoregion'] == ecoregion]
            corr_elev = spearmanr(ecoregion_df['elevation'], ecoregion_df['valid_trend'])
            corr_slope = spearmanr(ecoregion_df['slope'], ecoregion_df['valid_trend'])
            corr_aspect_n = spearmanr(ecoregion_df['aspect_n'], ecoregion_df['valid_trend'])
            corr_aspect_e = spearmanr(ecoregion_df['aspect_e'], ecoregion_df['valid_trend'])
            local_r.append([ecoregion, corr_elev.statistic, corr_slope.statistic, corr_aspect_n.statistic, corr_aspect_e.statistic])
        except Exception as e:
            print(f"Error for ecoregion {ecoregion}: {e}")
            local_r.append([ecoregion, None, None, None, None])

    r_topo_lsp = pd.DataFrame(local_r, columns=['ecoregion', 'elevation', 'slope', 'aspect_n', 'aspect_e']).round(2)
    new_row_data = r_topo_lsp[['elevation', 'slope', 'aspect_n', 'aspect_e']].abs().mean().round(2)
    new_row_data['ecoregion'] = 'avg abs r'
    r_topo_lsp = pd.concat([r_topo_lsp, pd.DataFrame([new_row_data])], ignore_index=True)

    r_topo_lsp.to_csv(os.path.join(base_outdir, "spmn_r_topographic_" + metric + ".csv"), index=False)
    print(r_topo_lsp)
    print("Correlations for " + metric + " saved to " + os.path.join(base_outdir, "correlations_topographic_" + metric + ".csv"))

    ecoregion  elevation  slope  aspect_n  aspect_e
0       81003       0.09  -0.05     -0.01     -0.01
1       40115       0.09  -0.00      0.02     -0.03
2       40301       0.05  -0.02      0.03     -0.01
3       40403      -0.06   0.06      0.06     -0.06
4       40401       0.06   0.05     -0.02     -0.01
5       40166        NaN    NaN       NaN       NaN
6       81021       0.03  -0.06      0.04     -0.01
7       40701       0.35   0.21     -0.03     -0.02
8       40501       0.07  -0.01     -0.02     -0.02
9       40502       0.23   0.01     -0.00     -0.02
10      40120        NaN    NaN       NaN       NaN
11  avg abs r       0.11   0.05      0.03      0.02
Correlations for sos saved to ../Data/Processed/Correlation_Analysis/Topographic_Factors/correlations_topographic_sos.csv
    ecoregion  elevation  slope  aspect_n  aspect_e
0       81003      -0.06  -0.00     -0.04      0.00
1       40115      -0.09  -0.10      0.02      0.01
2       40301       0.04   0.04     -0.01     